In [ ]:
import tensorflow as tf
import numpy as np
import datetime

In [ ]:
!lscpu

In [ ]:
!cat /proc/cpuinfo

In [ ]:
# anomaly detection model
# https://github.com/mlcommons/tiny_results_v0.5/blob/main/closed/reference/code/training/anomaly_detection/keras_model.py
def get_model(inputDim):
    """
    define the keras model
    the model based on the simple dense auto encoder 
    (128*128*128*128*8*128*128*128*128)
    """
    inputLayer = tf.keras.layers.Input(shape=(inputDim,))

    h = tf.keras.layers.Dense(128)(inputLayer)
    h = tf.keras.layers.BatchNormalization()(h)
    h = tf.keras.layers.Activation('relu')(h)

    h = tf.keras.layers.Dense(128)(h)
    h = tf.keras.layers.BatchNormalization()(h)
    h = tf.keras.layers.Activation('relu')(h)

    h = tf.keras.layers.Dense(128)(h)
    h = tf.keras.layers.BatchNormalization()(h)
    h = tf.keras.layers.Activation('relu')(h)

    h = tf.keras.layers.Dense(128)(h)
    h = tf.keras.layers.BatchNormalization()(h)
    h = tf.keras.layers.Activation('relu')(h)

    h = tf.keras.layers.Dense(8)(h)
    h = tf.keras.layers.BatchNormalization()(h)
    h = tf.keras.layers.Activation('relu')(h)

    h = tf.keras.layers.Dense(128)(h)
    h = tf.keras.layers.BatchNormalization()(h)
    h = tf.keras.layers.Activation('relu')(h)

    h = tf.keras.layers.Dense(128)(h)
    h = tf.keras.layers.BatchNormalization()(h)
    h = tf.keras.layers.Activation('relu')(h)

    h = tf.keras.layers.Dense(128)(h)
    h = tf.keras.layers.BatchNormalization()(h)
    h = tf.keras.layers.Activation('relu')(h)

    h = tf.keras.layers.Dense(128)(h)
    h = tf.keras.layers.BatchNormalization()(h)
    h = tf.keras.layers.Activation('relu')(h)

    h = tf.keras.layers.Dense(inputDim)(h)

    return tf.keras.models.Model(inputs=inputLayer, outputs=h)


# https://github.com/mlcommons/tiny_results_v0.5/blob/main/closed/reference/code/training/anomaly_detection/00_train.py#L197
#model = keras_model.get_model(param["feature"]["n_mels"] * param["feature"]["frames"])

# params: https://github.com/mlcommons/tiny_results_v0.5/blob/main/closed/reference/code/training/anomaly_detection/baseline.yaml

INPUT_DIM = 128 * 5
ad_model = get_model(INPUT_DIM)
#ad_model.summary()

In [ ]:
N_SAMPLES = 32_768
rng = np.random.default_rng()
samples = rng.random((N_SAMPLES, INPUT_DIM), dtype=np.float32)

timings = {}
for batch_size  in [8, 16, 32, 64, 128, 256, 512]:
    start = datetime.datetime.now()
    ds = (
        tf.data.Dataset.from_tensor_slices(samples)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )    
    results = ad_model.predict(ds)
    end = datetime.datetime.now()
    timings[batch_size] = end - start
timings

In [ ]:
mobilenet = tf.keras.applications.MobileNet()

In [ ]:
N_IMAGES = 512
rng = np.random.default_rng()
images = rng.integers(0, 255, size=(N_IMAGES, 224,224,3), dtype=np.uint8)

timings = {}
for batch_size  in [8, 16, 32, 64, 128]:
    start = datetime.datetime.now()
    ds = (
        tf.data.Dataset.from_tensor_slices(tf.keras.applications.mobilenet.preprocess_input(images))
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )    
    results = mobilenet.predict(ds)
    end = datetime.datetime.now()
    timings[batch_size] = end - start

In [ ]:
timings